# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации

In [487]:
# импорт библиотеки pandas и открытие файла
import pandas as pd
df = pd.read_csv('/Users/pomakhin/Desktop/Data_analysis/Python/Progects/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [488]:
df.info() # получение общей информации о файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Типы данных в столбцах: `float64`, `int64` и `object`.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — образование клиента
* `education_id` — идентификатор образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — доход в месяц
* `purpose` — цель получения кредита

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.
Кроме того, в столбце `days_employed` уже на предпросмотре видны отрицательные значения и анамально большие величины.

Кроме того, видны дубли в столбце `education`, вызванные разными регистрами написания. На этапе предобработки данных проверим наличие подобных неявных дубликатов во всех столбцах таблицы и в случае, если их доля критическая (более 5%) - избавимся от них одним из методов обраюотки дубликатов.

**Вывод**

В каждой строке таблицы — данные о клиенте банка. Часть колонок описывает самого клиента: пол, возраст, образование, семейное положение, количество детей в семье. Часть данных отражает трудовой стаж клиента, тип занятости и уровень ежемесячного дохода. Остальные данные показывают имел ли место факт задолженности по возврату кредитов клиентом, а также цель получения кредита.

Предварительно можно утверждать, что, данных достаточно для проверки того, влияет ли семейное положение и количество детей клиента и цели кредита на факт погашения кредита в срок. 

Прежде чем начинать анализ, необходима предобработка данных.

## Предобработка данных

### Обработка пропусков

В ходе предврительного изучения данных таблицы, были замечены анамалии в данных в столбце `days_employed` (отрицательные и анамально высокие значения). Проверим остальные столбцы.

In [489]:
# проверка уникальных значений в столбце 'children' с использованием value_counts (подсчет уникальных значений столбца), 
# так сразу можно оценить долю аномалий в общем объеме данных
df['children'].value_counts()   

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Аномалии в столбце `children` (значения "-2" и "20") занимают меньше 5% в данных, поэтому ими можно пренебречь - удалим их (заменим на 0).

In [514]:
# для этого используем простой для понимания метод loc, позволяющий использовать несколько условий
df.loc[(df['children'] < 0) | (df['children'] == 20), 'children'] = 0
df['children'].value_counts()  # проверка

0    14212
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Отрицательные значения в столбце  `days_employed` не влияют на текущий проект, но для учебных целей проверим долю этих значений в общей массе и приведем к положительным значениям.

In [491]:
# проверка доли отрицательных значений в столбце 'days_employed' 
# с истользованием value_counts (подсчет уникальных значений столбца) с использованием условия 
display((df['days_employed'] < 0).value_counts())

# сразу приведем отрицательные значения стажа к положительным 
# тут использован самый логичный для этого метод модуля abs при помощи apply, 
# который легко позволяет применять методы ко всему столбцу 
df['days_employed'] = df['days_employed'].apply(abs)

display((df['days_employed'] < 0).value_counts()) # проверка

True     15906
False     5619
Name: days_employed, dtype: int64

False    21525
Name: days_employed, dtype: int64

In [492]:
# проверим уникальные значения столбца 'dob_years'
df['dob_years'].sort_values(ascending=True).unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

В столбце  `dob_years` выявлены нулевые значения возраста. Объем значений не превышает 5% от общих данных и их можно удалить. Тут для учебных целей заполним пропуски медианными значениями с учетом ктегорий клиентов.

In [493]:
# приведение нулевых значений возрастра клиента к медианным
# использован loc в связке с median
df.loc[df['dob_years'] == 0, 'dob_years'] = df['dob_years'].median()
print('Количество нулевых значений возраста клиента:', df[df['dob_years'] == 0]['dob_years'].count())   # проверка

Количество нулевых значений возраста клиента: 0


In [494]:
# проверим уникальные значения столбца 'gender'
display(df['gender'].value_counts())

# удалим значение XNA (оно всего 1 и его удаление не существенно)
df = df[df['gender'] != 'XNA']
display(df['gender'].value_counts()) # проверка

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

F    14236
M     7288
Name: gender, dtype: int64

In [495]:
# проверка уникальных крайних значений в столбце 'total_income'
df['total_income'].sort_values(ascending=True).unique()

array([  20667.26379327,   21205.28056622,   21367.64835649, ...,
       2200852.2102589 , 2265604.02872274,              nan])

* Колонка `children` (количество детей у клиента) содержит отрицательное значение и экстремально высокое (20). Вероятнее всего это ошибки ввода: клавищи 0 и 20 близко расоложены на клавиатуре, а отрицательные значения вероятно результат выгрузки из разных баз - аномалии удалены (приведены к 0).
* Колонка `days_employed` (стаж клиента в днях) также содержит отрицательные значение и экстремально высокие. Отрицательные значения вероятнее всего техническая ошибка: значения приведены к положительным. Аномально большие значения (величина данного показателя более 30000 дней не логична, так как такой стаж не реалистичен) - вероятно значения были выгружены в часах. Данные этого показателя не участвуют в текущем проекте при проработке гиотез и ими можно принебречь.
* Колонка `dob_years` (возраст клиента) содержит нулевые значения, что является пропуском данных. Доля пропусков менее 5% и их можно удалить, но в учебных целях пропуски заполнены медианным значением.
* Также удалено некорректное значение в столбце `gender` - вероятно техническая ошибка - результат автозаполнения пропуска.

Посчитаем, сколько в таблице пропущенных значений.

In [496]:
df.isna().sum() # сочетание isna и sum дает общее количество пропусков по каждому столбцу

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [497]:
# Заполним пропуски в days_employed медианными значениями, но учтем также возраст клиента
# для этого в цикле пропишем условия с помощью loc и подсчитаем медиану
for years in df['dob_years'].unique():
    median = df.loc[df['dob_years'] == years, 'days_employed'].median()
    df.loc[(df['days_employed'].isna()) & (df['dob_years'] == years), 'days_employed'] = median

print('Количество пропусков в days_employed:', df['days_employed'].isna().sum()) # проверка

Количество пропусков в days_employed: 0


In [498]:
# Заполним пропуски в total_income медианными значениями, но учтем также тип занятости клиента
for types in df['income_type'].unique():
    median = df.loc[df['income_type'] == types, 'total_income'].median()
    df.loc[(df['total_income'].isna()) & (df['income_type'] == types), 'total_income'] = median

print('Количество пропусков в total_income:', df['total_income'].isna().sum()) # проверка

Количество пропусков в total_income: 0


**Вывод**

В результате обработки пропусков, а также аномальных значений в данных:
* в колонках `children` и `days_employed`  отрицательные значения были приведены к положительным/заменены на 0
* пропуски в колонках `dob_years`, `days_employed` и `total_income` были заменены медианными значениями для связок категорий данных.

### Замена типа данных

In [499]:
# Выполним замену типов данных в days_employed и total_income с вещественного на целочисленный при помощи метода astype, 
# который позволяет изменять значения столбца на любой тип данных
df['days_employed'] = df['days_employed'].astype(int)
display(df['days_employed'].head()) # проверка

df['total_income'] = df['total_income'].astype(int)
df['total_income'].head() # проверка

0      8437
1      4024
2      5623
3      4124
4    340266
Name: days_employed, dtype: int64

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

**Вывод**

Для стодбцов `days_employed` и `total_income` выполнили замену типа данных с вещественного на целочисленный для удобства работы с данными и малой погрешности ввиду отбрасывания дробной части.

### Обработка дубликатов

Проверим данные на неявные дубликаты.

In [500]:
# сочетание drop_duplicates и sort_values позволяет покатать уникальные значения в отсортированном виде
display(df[['education']].drop_duplicates().sort_values(by='education')) 

# исправим выявленные неявные дубликаты путем приведения строк в колонке к нижнему регистру специальным для этого методом str.lower
df['education'] = df['education'].str.lower()
df[['education']].drop_duplicates() # проверка

,education
8,ВЫСШЕЕ
62,Высшее
797,НАЧАЛЬНОЕ
134,НЕОКОНЧЕННОЕ ВЫСШЕЕ
2817,Начальное
376,Неоконченное высшее
7,СРЕДНЕЕ
2,Среднее
4170,УЧЕНАЯ СТЕПЕНЬ
2963,Ученая степень


,education
0,высшее
1,среднее
13,неоконченное высшее
31,начальное
2963,ученая степень


In [501]:
# аналогичная проверка столбца family_status
df[['family_status']].drop_duplicates()

,family_status
0,женат / замужем
4,гражданский брак
18,вдовец / вдова
19,в разводе
24,Не женат / не замужем


In [502]:
# аналогичная проверка столбца income_type
df[['income_type']].drop_duplicates()

,income_type
0,сотрудник
4,пенсионер
5,компаньон
26,госслужащий
3133,безработный
5936,предприниматель
9410,студент
20845,в декрете


In [503]:
# Удалим явные дубликаты в таблице с удалением старых индексов и формированием новых
# с исользованием связки drop_duplicates и reset_index, что позволяет перезаписать старые индексы

print('Количество дубликатов до обработки:', df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True) 
print('Количество дубликатов после обработки:', df.duplicated().sum())

Количество дубликатов до обработки: 72
Количество дубликатов после обработки: 0


**Вывод**

Методом `drop_duplicates` удалили явные строки-дубликаты в таблице. Причина появления которых вероятно техническая, так как строки полностью повторяются.

Проверили на наличие неявных дубликтов столбцы с категориальными значениями путем вывода уникальных значений: дубликаты присутствовали только в колонке `education` - вероятно результат объединения разных баз или ошибок ввода. Дубликаты были удалены путем приведения к единому регистру. 

Столбец `purpose` проверим на следующем шаге в ходе его лемматизации для дальнейшей категоризации.

### Лемматизация

Для целей дальнейшей категоризации данных для проверки гипотез - лемматизируем столбец `purpose`.

In [504]:
# импортируем библиотеку для лемматизации
from pymystem3 import Mystem
m = Mystem() 
# импортируем Counter для подсчета частоты появления слов
from collections import Counter

# находим уникальные значения столбца (метод unique) и переводим их из списка в строку для дальнейшей лемматизации 
# (методом join склеиваем значения в строчку с использованием разделтелем пробел)
purpose_unique =  ' ' .join(df['purpose'].unique())

# лемматизируем полученные значения столбца 'purpose'
lemmas = m.lemmatize(purpose_unique)
# считаем частоту появления слов
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


**Вывод**

По итогам лемматизации самые часто встречающиеся существительные:
* недвижимость (при категоризации объеденим с категорией "жильё"; цели с леммой "операция" также относятся к категории жильё/недвижимость)
* автомобиль
* образование
* свадьба
* строительство

Далее сформируем соответствующие категории целей кредита - добавим также катагорию 'прочее' для остальных менее частовстречающихся целей.

### Категоризация данных

Выделение данных в категории необходимо для ответа на поставленные вопросы.

Начнем категоризацию со столбца purpose - используем категории целей кредита, полученные в ходе лемматизации.

In [505]:
# Определим функцию для лемматицазии и категоризации данных о целях получения кредита
# функция будет лемматизировать последовательно каждую ячейку столбца и проверять наличие категорий в лемматизированных значениях
def purpose_categorize(cells_values):
    cells_values_lemmas = m.lemmatize(cells_values) 
    if 'недвижимость' in cells_values_lemmas: 
        return 'недвижимость'
    if  'жилье' in cells_values_lemmas: 
        return 'недвижимость'
    if 'автомобиль' in cells_values_lemmas: 
        return 'авто'
    if 'образование' in cells_values_lemmas: 
        return 'образование'
    if 'свадьба' in cells_values_lemmas: 
        return 'свадьба'
    if 'строительство' in cells_values_lemmas: 
        return 'строительство'      
    return 'прочее'
        
# Выполним функцию для столбца 'purpose', применив метод apply, который позволяет применить функцию для всего столбца
# в результате создадим новый столбец 'purpose_category' с категориями целей получения кредита
df['purpose_category'] = df['purpose'].apply(purpose_categorize)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,авто
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [506]:
# Добавим категории возраста клиента для этого создадим функцию, которая определяет категорию сравнивая с заданными возрастными рамками
def age_category_def(age_value):
    if age_value <= 25:
        return '18-25'
    if age_value <= 35:
        return '26-35'
    if age_value <= 45:
        return '36-45'
    else: 
        return '46+'
    
# Применим функцию к столбцу dob_years, создав новый стодбец age_category
df['age_category'] = df['dob_years'].apply(age_category_def)

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,age_category
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,36-45
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,авто,36-45
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,26-35
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,26-35
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,46+


Для отображения имеющихся в данных категорий - построим фреймы из их уникальных значений для понимания перечня категорий.

In [507]:
# Отобразим категории уровня образования клиентов
# в сочетании drop_duplicates и set_index установим уникальные значения и индексы для них из имеющихся данных в колонках
df[['education_id', 'education']].drop_duplicates().set_index('education_id')

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


In [508]:
# Отобразим категории семейного положения клиентов
df[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')

,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


Дополнительно введем категории уровня дохода клиентов для проработки гиотез.

In [509]:
# Добавим категории в зависимости от уровня дохода клиента
# создадим функцию, которая определяет категорию, сравнивая с заданными рамками уровня дохода
def income_level_def(income_value, min_level, mid_level):
    if income_value <= min_level:
        return 'низкий'
    if income_value <= mid_level:
        return 'средний'
    else: 
        return 'высокий'
    
# Применим функцию к столбцу total_income (методом apply с исользованием параметра args, чтобы отдать функции доп. значения)
# создадим новый стодбец income_category и зададим уровни дохода для категоризации
df['income_level'] = df['total_income'].apply(income_level_def, args=(100000, 200000))

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,age_category,income_level
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,36-45,высокий
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,авто,36-45,средний
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,26-35,средний
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,26-35,высокий
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,46+,средний


**Вывод**

Для целей проверки поставленных гипотез, были категоризованы данные:
* цели получения кредита клиентами банка сгрупированы в 4 крупные категории на основании лемматизации данных колонки с целями кредита `purpose`
* уровень дохода клиентов был сгрупирован в 3 категории по заданным уровням (уровни взяты произвольно в связи с тем, что данные о доходах учебные)
* были отображены имеющиеся в табдице категории образования и семейного положения, которые необходимы для анализа и проработки поставленных гиотез.

## Шаг 3. Ответьте на вопросы

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [510]:
# Для ответа на поставленный вопрос построим сводную таблицу методом pivot_table
# сгрупируем таблицу по количеству детей и подсчитаем количество возвратов/невозвратов по каждой группе
report = df.pivot_table(index='children', columns='debt', values='total_income', aggfunc='count')

report.columns = ['debt', 'no_debt']
# подсчитаем также долю возврата кредита и отсортируем по этому значению в порядке убывания
report['repayment_%'] = report['debt'] / (report['debt'] + report['no_debt']) 
report.sort_values(by='repayment_%', ascending=False)

,debt,no_debt,repayment_%
children,,,
0,13140.0,1072.0,0.924571
3,303.0,27.0,0.918182
1,4364.0,444.0,0.907654
2,1858.0,194.0,0.905458
4,37.0,4.0,0.902439
5,9.0,NaN,NaN


**Вывод**

* Полученные данные показывают, что наибольшая доля возврата кредита среди клиентов без детей.
* При этом самая низкая доля возврата кредита у клиентов с 4, 2 и 1 ребенком.

Можно предположить, что клиенты без детей чаще возвращают кредит без задержек, чем клиенты с детьми. Однако, различия не существенны.
***

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [511]:
# построим сводную таблицу методом pivot_table
# сгрупируем таблицу по категориям семейного положения клиентов, а также добавим возрастную категорию
# и подсчитаем количество возвратов/невозвратов по каждой категории
report = df.pivot_table(index=['family_status', 'age_category'], columns='debt', values='total_income', aggfunc='count')

report.columns = ['debt', 'no_debt']
# подсчитаем также долю возврата кредита и отсортируем по этому значению в порядке убывания
report['repayment_rate'] = report['debt'] / (report['debt'] + report['no_debt']) 
report.sort_values(by='repayment_rate', ascending=False)

debt  no_debt  repayment_rate
family_status         age_category                                 
женат / замужем       46+           4975.0    302.0        0.942771
в разводе             36-45          342.0     21.0        0.942149
вдовец / вдова        36-45           87.0      6.0        0.935484
                      46+            791.0     55.0        0.934988
Не женат / не замужем 46+            793.0     56.0        0.934040
в разводе             46+            560.0     40.0        0.933333
гражданский брак      46+           1463.0    115.0        0.927123
женат / замужем       36-45         3233.0    272.0        0.922397
Не женат / не замужем 36-45          525.0     54.0        0.906736
женат / замужем       18-25          390.0     41.0        0.904872
в разводе             26-35          190.0     20.0        0.904762
гражданский брак      18-25          274.0     29.0        0.904290
женат / замужем       26-35         2810.0    316.0        0.898912
гражданский брак      36-45         1058.0    122.0        0.896610
вдовец / вдова        26-35           17.0      2.0        0.894737
гражданский брак      26-35          966.0    122.0        0.887868
Не женат / не замужем 26-35          801.0    107.0        0.882159
                      18-25          417.0     57.0        0.879747
в разводе             18-25           18.0      4.0        0.818182
вдовец / вдова        18-25            1.0      NaN             NaN

**Вывод**

Данные показывают, что самые высокие показатели возврата кредита у клиентов со статусом "женат / замужем", "в разводе" и "вдова/вдовец". При этом, если учитывать еще и возраст клиентов, то наблюдается зависисмость: клиенты более высоких возрастных категорий в среднем имеют меньше задолженностей по кредитам, чем клиенты более низких возрастных категорий.
***

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [512]:
# построим сводную таблицу методом pivot_table
# сгрупируем таблицу по уровням дохода клиентов
# и подсчитаем количество возвратов/невозвратов по каждой категории
report = df.pivot_table(index='income_level', columns='debt', values='total_income', aggfunc='count')

report.columns = ['debt', 'no_debt']
# подсчитаем также долю возврата кредита и отсортируем по этому значению в порядке убывания
report['repayment_rate'] = report['debt'] / (report['debt'] + report['no_debt']) 
report.sort_values(by='repayment_rate', ascending=False)

,debt,no_debt,repayment_rate
income_level,,,
высокий,4708,358,0.929333
низкий,4109,354,0.920681
средний,10894,1029,0.913696


**Вывод**

Самый высокий показатель возврата кредита у клиентов с высоким уровнем дохода, самый низкий у клиентов со средним доходом. При этом в целом клиентов со средним уровнем дохода существенно больше остальных категорий и возможна статистическая погрешность. В целом различия не высоки.
***

4. Как разные цели кредита влияют на его возврат в срок?

In [513]:
# построим сводную таблицу методом pivot_table
# сгрупируем таблицу по категориям целей получения кредита
# и подсчитаем количество возвратов/невозвратов по каждой категории
report = df.pivot_table(index='purpose_category', columns='debt', values='total_income', aggfunc='count')

report.columns = ['debt', 'no_debt']
# подсчитаем также долю возврата кредита и отсортируем по этому значению в порядке убывания
report['repayment_rate'] = report['debt'] / (report['debt'] + report['no_debt']) 
report.sort_values(by='repayment_rate', ascending=False)

,debt,no_debt,repayment_rate
purpose_category,,,
недвижимость,10028,782,0.927660
свадьба,2137,186,0.919931
образование,3643,370,0.907800
авто,3903,403,0.906410


**Вывод**

Данные показывают, что самая большая доля возвратов кредитов, которые были взяты для целей из категории "недвижимость". Самый низкий показатель возвратов в категории "авто".

Можно дополнительно рассмотреть подробнее не только категории целей кредита, но и наример в связке с возрастом клиентов, так как вероятно кредиты на авто чаще берут более молодые клиенты, а кредиты на недвижимость клиенты более старшего возраста. Таким образом, можно проверить нет ли тут псевдо-зависимости доли возвратов кредитов от целей.

## Общий вывод

В ходе выполнения проекта были исследованы данные от банка — статистика о платёжеспособности клиентов. Данные были предобработаны:
* были обработаны и устранены пропуски в данных, так как они могли исказить результаты исследования
* выявлены и устранены явные дубликаты данных, а также скорректированы неявные
* были скорректированы типы данный для колличественных значений
* выолнена категоризация данных для удобства проверки поставленных гиотез (в том числе одна из групп категорий была сформирована на базе лемматизации данных).

На основании полученных данных удалось проверить 4 гипотезы:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

Данные показали, что клиенты без детей чаще возвращают кредит без задержек, чем клиенты с детьми. Однако, различия не существенны и возможно, следует проверить гиотезу на бОльших объемах данных или в связке с дригими критериями.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Клиенты со статусами:  "женат / замужем", "в разводе" и "вдова/вдовец" чаще возвращают кредит без задержек. Однако, придобавлении в анализ категории возраста клиентов выявлена более четкая связь: более молодые клиенты чаще имеют невозвраты по кредитам, в сравнении с клиентами более старшего возраста.

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Выявлено, что клиенты с высоким доходом чаще возвращают кредиты. При этом различия не велики и вероятно необходимо проверить прочие влияния в связке наример с типом занятости клиентов.

4. Как разные цели кредита влияют на его возврат в срок?

Для целей из категории "недвижимость" доля возвратов больше, при этом самая маленькая доля возвратов в категории "авто". Эти данные также можно проверить в связке с другими критериями (например возрастом клиента) для выявления комлексных взаимосвязей и доолнительной проверки гипотезы. 

**Итог**

Общие ответы на поставленные вопросы получены: взаимосвязи есть, однако вероятно, требуется более комлексная проработка взаимоствязей между различными критериями.
